In [2]:
from cltk import NLP

# Initialize the Latin language processor
cltk_nlp = NLP(language="lat")
cltk_nlp.analyze(text="Gallia est omnis divisa in partes tres")

e:\Miniconda\Enviroments\transformer-lang\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


‎𐤀 CLTK version '1.1.6'.
Pipeline for language 'Latin' (ISO: 'lat'): `LatinNormalizeProcess`, `LatinStanzaProcess`, `LatinEmbeddingsProcess`, `StopsProcess`, `LatinLexiconProcess`.


Doc(language='lat', words=[Word(index_char_start=None, index_char_stop=None, index_token=0, index_sentence=0, string='Gallia', pos=noun, lemma='Gallia', stem=None, scansion=None, xpos='A1|grn1|casA|gen2', upos='NOUN', dependency_relation='nsubj', governor=3, features={Case: [nominative], Gender: [feminine], Number: [singular]}, category={F: [neg], N: [pos], V: [neg]}, stop=False, named_entity=None, syllables=None, phonetic_transcription=None, definition=''), Word(index_char_start=None, index_char_stop=None, index_token=1, index_sentence=0, string='est', pos=auxiliary, lemma='sum', stem=None, scansion=None, xpos='N3|modA|tem1|gen6', upos='AUX', dependency_relation='cop', governor=3, features={Mood: [indicative], Number: [singular], Person: [third], Tense: [present], VerbForm: [finite], Voice: [active]}, category={F: [pos], N: [neg], V: [pos]}, stop=True, named_entity=None, syllables=None, phonetic_transcription=None, definition='sum\n\n (2d pers. es, or old ēs; old subj praes. siem,\n  

In [3]:
from cltk.lemmatize import LatinBackoffLemmatizer

lemmatizer = LatinBackoffLemmatizer()

def get_lemma(word):
    lemmatized_word = lemmatizer.lemmatize([word])
    return lemmatized_word[0][1]



In [4]:
latin_numerals = {
    'I': 1,
    'V': 5,
    'X': 10,
    'L': 50,
    'C': 100,
    'D': 500,
    'M': 1000
}

def translate_latin_to_normal(latin_numeral):
    total = 0
    prev_value = 0

    for numeral in reversed(latin_numeral):
        value = latin_numerals.get(numeral, 0)

        if value >= prev_value:
            total += value
        else:
            total -= value

        prev_value = value

    return total

In [30]:
class VariableDeclarationException(Exception):
    pass

class IfStatementException(Exception):
    pass

In [6]:
stack = []
programmCounter = -1

In [44]:
def findByCase(target_case, words):
    return [entry["lemma"] for entry in words if entry["pos"] == "NOUN" and target_case in str(entry['features']["Case"])]

def findNumber(words):
    return [entry["lemma"] for entry in words if entry["pos"] == "NUM"]

def findObjektAttributes(stack, variableName):
    for i in stack:
        if i["object"] == variableName:
            return i

In [46]:
def findVariableDefinitions(words) -> bool: # auf doppelte Variablen prüfen
    #for word in words:
    if any(entry["lemma"]=="appero" for entry in words):
        target_case = '[nominative]'
        matching_words = [entry['word'] for entry in words if entry["pos"] == "NOUN" and target_case in str(entry['features']["Case"])]
        if len(matching_words) == 1:
            stack.append({"object": matching_words[0], "attributes": []})
            return True
        elif len(matching_words) > 1:
            raise VariableDeclarationException("More than one variable found")
    return False

def findVariableAssignments(words) -> bool:
    #words = [{"word": word.string, "pos": word.upos, "features": word.features, "lemma": word.lemma} for word in analysis.words]
    foundNumbers = [entry['word'] for entry in words if entry["pos"] == "NUM"]
    if len(foundNumbers) == 1:
        target_case = 'nominative'
        foundVariable = findByCase(target_case, words)
        if len(foundVariable) == 1:
            variable = foundVariable[0]
            for i in stack:
                if i["object"] == variable:
                    target_case = 'accusative'
                    foundAttributes = findByCase(target_case, words)
                    if len(foundAttributes) == 1:
                        i["attributes"].append({"name": get_lemma(foundAttributes[0]), "type": "number", "value": translate_latin_to_normal(foundNumbers[0])})
                    
                    
                    # i["value"] = translate_latin_to_normal(foundNumbers[0])
                    # i["type"] = "number"
                    break
        return True
    return False



def ifCondition(words) -> bool:
    beforeComma = []
    afterComma = []
    foundComma = False
    if not any(entry["lemma"]=="si" for entry in words):
        return False
    for word in words:
        if word["word"] == ",":
            foundComma = True
        elif not foundComma:
            beforeComma.append(word)
        else:
            afterComma.append(word)
    if len(beforeComma) == 0 or len(afterComma) == 0:
        raise IfStatementException("If statement is missing a segment")
    
    
    
    target_case = 'nominative'
    foundVariable = findByCase(target_case, beforeComma)
    print(foundVariable)
    target_case = 'accusative'
    foundAttributes = findByCase(target_case, beforeComma)
    print(foundAttributes)
    foundNumber = findNumber(beforeComma)
    print(foundNumber)
    if not (len(foundVariable) == 1 and len(foundAttributes) == 1 and len(foundNumber) == 1):
        raise IfStatementException("If statement got to many or to few arguments")
    print(findObjektAttributes(stack, foundVariable[0]))
    for attribute in findObjektAttributes(stack, foundVariable[0])["attributes"]:
        if attribute["name"] == get_lemma(foundAttributes[0]):
            if attribute["value"] == translate_latin_to_normal(foundNumber[0]):
                print("true")
                return True
    
    
    
    
    
    
    
    
    
    return False
    
    
    
    # if any(entry["lemma"]=="si" for entry in words):
    #     target_case = '[nominative]'
    #     foundVariable = [entry['word'] for entry in words if entry["pos"] == "NOUN" and target_case in str(entry['features']["Case"])]
    #     if len(foundVariable) == 1:
    #         variable = foundVariable[0]
    #         for i in stack:
    #             if i["name"] == variable:
    #                 print(i["value"])

In [48]:
stack = []
programmCounter = -1


with open("text.txt") as f:
    text = f.readlines()
    for i in range(len(text)):
        programmCounter += 1
        line = text[i]
        words = []
        analysis = cltk_nlp.analyze(text=line)
        words = [{"word": word.string, "pos": word.upos, "features": word.features, "lemma": word.lemma} for word in analysis.words]
        
        
        if ifCondition(words):
            continue
        elif findVariableDefinitions(words):
            continue
        elif findVariableAssignments(words):
            continue

    print(stack)

['Herkules']
['asinus']
['X']
{'object': 'Herkules', 'attributes': [{'name': 'asinus', 'type': 'number', 'value': 9}, {'name': 'nauta', 'type': 'number', 'value': 5}]}
[{'object': 'Herkules', 'attributes': [{'name': 'asinus', 'type': 'number', 'value': 9}, {'name': 'nauta', 'type': 'number', 'value': 5}]}, {'object': 'Flavia', 'attributes': [{'name': 'collega', 'type': 'number', 'value': 3}]}]
